In [1]:
import pandas as pd
from pathlib import Path
import uuid

In [2]:
fincausal_task1_train_path = Path('/media/sarthak/HDD/data_science/fnp_resources/data/task1/iteration_5/train.csv')
semeval_all_data_path = Path('/media/sarthak/HDD/data_science/fnp_resources/data/extra/semeval.csv')

In [18]:
fincausal_task1_train_path / 'a'

PosixPath('/media/sarthak/HDD/data_science/fnp_resources/data/task1/iteration_5/train.csv/a')

In [3]:
semeval_all_data_df = pd.read_csv(semeval_all_data_path)
semeval_all_data_df.head()

,sentence,relation,causal
0,The system as described above has its greatest...,Component-Whole,0
1,The child was carefully wrapped and bound into...,Other,0
2,The author of a keygen uses a disassembler to ...,Instrument-Agency,0
3,A misty ridge uprises from the surge.,Other,0
4,The student association is the voice of the un...,Member-Collection,0


In [4]:
semeval_all_data_df.shape

(10717, 3)

In [5]:
fincausal_task1_train_df = pd.read_csv(fincausal_task1_train_path)
fincausal_task1_train_df.head()

,Index,Text,Gold,unique_id
0,268.00017,"In turn, the Liberal promise could benefit th...",0,5e9f149f-e1c3-43b8-a4f0-2614ca2bfd42
1,93.00006,Research analysts expect Mackinac Financial t...,0,57b71014-8f81-4d77-a1e2-aee69c36be92
2,471.00003,The law firm Kessler Topaz Meltzer & Check LL...,0,661e6bf7-ffbf-4581-acff-8c246ecac51a
3,320.00001,News The UK Civil Aviation Authority has lau...,0,a6a80d24-dbc0-404d-9f71-e907524e88ec
4,521.00016,The proportion of middle-skill jobs for worke...,0,9657941b-aae0-4a9c-b1cd-8c7433f3b0bf


In [6]:
fincausal_task1_train_df.shape

(15576, 4)

In [7]:
# what's the distribution of label?
causal_distribution = fincausal_task1_train_df.Gold.value_counts() / len(fincausal_task1_train_df)
causal_distribution

0    0.927902
1    0.072098
Name: Gold, dtype: float64

In [8]:
# how many causal can I take from semeval?
semeval_noncausal = semeval_all_data_df.loc[semeval_all_data_df.causal == 0]
semeval_causal = semeval_all_data_df.loc[semeval_all_data_df.causal == 1]

semeval_causal_resampled = semeval_causal.sample(n=int((causal_distribution[1]*100*len(semeval_noncausal))/(100-(causal_distribution[1]*100))))

semeval_resampled = pd.concat([semeval_noncausal, semeval_causal_resampled])
semeval_resampled.causal.value_counts() / len(semeval_resampled)

0    0.927929
1    0.072071
Name: causal, dtype: float64

In [9]:
# combine both the datasets
semval_resampled_formatted_df = pd.DataFrame({
    'Index': None,
    'Text': semeval_resampled.sentence,
    'Gold': semeval_resampled.causal,
    'unique_id': [str(uuid.uuid4()) for _ in range(len(semeval_resampled))]
})
semval_resampled_formatted_df.head()

,Index,Text,Gold,unique_id
0,None,The system as described above has its greatest...,0,fea85888-9873-4814-b184-f1d1b5cade96
1,None,The child was carefully wrapped and bound into...,0,b6616479-ad1e-4386-abdf-02e392ec3978
2,None,The author of a keygen uses a disassembler to ...,0,504f321c-ecc9-4d8f-af37-41d8f24d8c3f
3,None,A misty ridge uprises from the surge.,0,6ca25b88-7c63-4b64-8b20-8762b7ee68ff
4,None,The student association is the voice of the un...,0,27cb3d7d-5348-4c45-9327-4efa0ec074b9


In [10]:
semval_resampled_formatted_df.Gold.value_counts() / len(semval_resampled_formatted_df)

0    0.927929
1    0.072071
Name: Gold, dtype: float64

In [11]:
fincausal_task1_train_combined_semeval = pd.concat([fincausal_task1_train_df, semval_resampled_formatted_df])
fincausal_task1_train_combined_semeval.shape

(25691, 4)

In [12]:
fincausal_task1_train_combined_semeval.Gold.value_counts() / len(fincausal_task1_train_combined_semeval)

0    0.927912
1    0.072088
Name: Gold, dtype: float64

In [13]:
fincausal_task1_train_combined_semeval.to_csv('/media/sarthak/HDD/data_science/fnp_resources/data/task1/iteration_5/train_with_semeval.csv', index=False)